In [1]:
from poker_dataset import PokerDataset
from poker_simulation import PokerProbabilityCalculator
from poker_baseline_model import get_bet_size
from helper import ACTION_NAMES
from tqdm import tqdm

import torch

In [2]:
filepath = r'C:\Users\kctak\Documents\Code\PokerAI\APS360 Progress\poker_data'
poker_dataset = PokerDataset(filepath, "IlxxxlI", balance_data=True)
print({action_name: len(indices) for action_name, indices in poker_dataset.output_indices.items()})

Processing Items: 100%|██████████| 50664/50664 [00:40<00:00, 1261.66it/s]


{'calls': 39175, 'raises': 39175, 'bets': 0, 'folds': 39175, 'checks': 39175, 'allin': 39175}


In [3]:
size_count = {'input_tensor': [], 'label_action_tensor': [], 'label_bet_size_tensor': []}
for i in tqdm(range(len(poker_dataset))):
    input_tensor, label_action_tensor, label_bet_size_tensor, baseline_inputs, game_idx = poker_dataset[i]
    size_count['input_tensor'].append(input_tensor.size())
    size_count['label_action_tensor'].append(label_action_tensor.size())
    size_count['label_bet_size_tensor'].append(label_bet_size_tensor.size())
    
print({key: set(value) for key, value in size_count.items()})

  0%|          | 0/195875 [00:00<?, ?it/s]

100%|██████████| 195875/195875 [00:00<00:00, 411162.03it/s]

{'input_tensor': {torch.Size([264])}, 'label_action_tensor': {torch.Size([11])}, 'label_bet_size_tensor': {torch.Size([1])}}


In [4]:
poker_dataset.games[9].rounds[0]

PREFLOP:
ElvenEyes raises 2.0 
Community Cards: []
Pot Size: 1.5
Round Call Amount: 0
Player Round Bet Size: 0
Player Stack: 111.3

gust folds 0 
Community Cards: []
Pot Size: 3.5
Round Call Amount: 2.0
Player Round Bet Size: 0
Player Stack: 47.0

VegetablesArentYummy folds 0 
Community Cards: []
Pot Size: 3.5
Round Call Amount: 2.0
Player Round Bet Size: 0
Player Stack: 273.59

StephCurry folds 0 
Community Cards: []
Pot Size: 3.5
Round Call Amount: 2.0
Player Round Bet Size: 0
Player Stack: 105.78

PANDAisEVIL folds 0 
Community Cards: []
Pot Size: 3.5
Round Call Amount: 2.0
Player Round Bet Size: 0
Player Stack: 101.0

AironVega folds 0 
Community Cards: []
Pot Size: 3.5
Round Call Amount: 2.0
Player Round Bet Size: 0
Player Stack: 101.7

IlxxxlI folds 0 
Community Cards: []
Pot Size: 3.5
Round Call Amount: 2.0
Player Round Bet Size: 0
Player Stack: 43.0

pineapplesand folds 0 
Community Cards: []
Pot Size: 3.5
Round Call Amount: 2.0
Player Round Bet Size: 0.5
Player Stack: 39.5

da

In [5]:
poker_dataset[9]

(tensor([ 0.4444,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

In [6]:
poker_calc = PokerProbabilityCalculator()
correct_count = 0
fold_count = 0
random_indices = torch.randperm(len(poker_dataset))[:5000]

label_count = {action_name: 0 for action_name in ACTION_NAMES}
bot_count = {action_name: 0 for action_name in ACTION_NAMES}

corr_count = 0
for i in tqdm(random_indices):
    input_tensor, label_action_tensor, label_bet_size_tensor, baseline_inputs, game_idx = poker_dataset[i]
    call_size = baseline_inputs['call_size']
    pot_size = baseline_inputs['pot_size']
    stack_size = baseline_inputs['stack_size']
    min_raise = baseline_inputs['min_raise']
    hole_cards = baseline_inputs['hole_cards']
    community_cards = baseline_inputs['community_cards']
    num_players = baseline_inputs['num_players']
    
    poker_calc.set(hole_cards, community_cards, num_players, num_simulations = 1000)
    win_rate, lose_rate, tie_rate, avg_tied_players = poker_calc.poker_probability_calculator()
    
    action_name, bet_size = get_bet_size(win_rate, lose_rate, tie_rate, avg_tied_players, pot_size, call_size, stack_size, min_raise, tight_factor=0.5)
    
    # label_action_tensor is a one hot tensor
    label_action_name = ACTION_NAMES[label_action_tensor.argmax().item()]
    #print(action_name, bet_size, "\t", label_action_name)
    label_count[label_action_name] += 1
    bot_count[action_name] += 1
    
    if action_name == label_action_name:
        corr_count += 1

print('label_count:', label_count)
print('bot_count:', bot_count)
print('corr_count:', corr_count)

100%|██████████| 5000/5000 [09:28<00:00,  8.79it/s]

label_count: {'small': 0, 'big': 0, 'calls': 960, 'caps': 0, 'raises': 962, 'bets': 0, 'folds': 1032, 'posts': 0, 'checks': 1053, 'straddle': 0, 'allin': 993}
bot_count: {'small': 0, 'big': 0, 'calls': 2, 'caps': 0, 'raises': 0, 'bets': 0, 'folds': 4998, 'posts': 0, 'checks': 0, 'straddle': 0, 'allin': 0}
corr_count: 1034


In [7]:
poker_calc = PokerProbabilityCalculator()
correct_count = 0
fold_count = 0

label_count = {action_name: 0 for action_name in ACTION_NAMES}
bot_count = {action_name: 0 for action_name in ACTION_NAMES}

corr_count = 0
for i in tqdm(random_indices):
    input_tensor, label_action_tensor, label_bet_size_tensor, baseline_inputs, game_idx = poker_dataset[i]
    call_size = baseline_inputs['call_size']
    pot_size = baseline_inputs['pot_size']
    stack_size = baseline_inputs['stack_size']
    min_raise = baseline_inputs['min_raise']
    hole_cards = baseline_inputs['hole_cards']
    community_cards = baseline_inputs['community_cards']
    num_players = baseline_inputs['num_players']
    
    poker_calc.set(hole_cards, community_cards, num_players, num_simulations = 1000)
    win_rate, lose_rate, tie_rate, avg_tied_players = poker_calc.poker_probability_calculator()
    
    action_name, bet_size = get_bet_size(win_rate, lose_rate, tie_rate, avg_tied_players, pot_size, call_size, stack_size, min_raise, tight_factor=0.2)
    
    # label_action_tensor is a one hot tensor
    label_action_name = ACTION_NAMES[label_action_tensor.argmax().item()]
    #print(action_name, bet_size, "\t", label_action_name)
    label_count[label_action_name] += 1
    bot_count[action_name] += 1
    
    if action_name == label_action_name:
        corr_count += 1

print('label_count:', label_count)
print('bot_count:', bot_count)
print('corr_count:', corr_count)

100%|██████████| 5000/5000 [09:13<00:00,  9.04it/s]

label_count: {'small': 0, 'big': 0, 'calls': 960, 'caps': 0, 'raises': 962, 'bets': 0, 'folds': 1032, 'posts': 0, 'checks': 1053, 'straddle': 0, 'allin': 993}
bot_count: {'small': 0, 'big': 0, 'calls': 73, 'caps': 0, 'raises': 0, 'bets': 1, 'folds': 4925, 'posts': 0, 'checks': 0, 'straddle': 0, 'allin': 1}
corr_count: 1074


In [8]:
poker_calc = PokerProbabilityCalculator()
correct_count = 0
fold_count = 0

label_count = {action_name: 0 for action_name in ACTION_NAMES}
bot_count = {action_name: 0 for action_name in ACTION_NAMES}

corr_count = 0
for i in tqdm(random_indices):
    input_tensor, label_action_tensor, label_bet_size_tensor, baseline_inputs, game_idx = poker_dataset[i]
    call_size = baseline_inputs['call_size']
    pot_size = baseline_inputs['pot_size']
    stack_size = baseline_inputs['stack_size']
    min_raise = baseline_inputs['min_raise']
    hole_cards = baseline_inputs['hole_cards']
    community_cards = baseline_inputs['community_cards']
    num_players = baseline_inputs['num_players']
    
    poker_calc.set(hole_cards, community_cards, num_players, num_simulations = 1000)
    win_rate, lose_rate, tie_rate, avg_tied_players = poker_calc.poker_probability_calculator()
    
    action_name, bet_size = get_bet_size(win_rate, lose_rate, tie_rate, avg_tied_players, pot_size, call_size, stack_size, min_raise, tight_factor=0.1)
    
    # label_action_tensor is a one hot tensor
    label_action_name = ACTION_NAMES[label_action_tensor.argmax().item()]
    #print(action_name, bet_size, "\t", label_action_name)
    label_count[label_action_name] += 1
    bot_count[action_name] += 1
    
    if action_name == label_action_name:
        corr_count += 1

print('label_count:', label_count)
print('bot_count:', bot_count)
print('corr_count:', corr_count)

100%|██████████| 5000/5000 [09:14<00:00,  9.02it/s]

label_count: {'small': 0, 'big': 0, 'calls': 960, 'caps': 0, 'raises': 962, 'bets': 0, 'folds': 1032, 'posts': 0, 'checks': 1053, 'straddle': 0, 'allin': 993}
bot_count: {'small': 0, 'big': 0, 'calls': 240, 'caps': 0, 'raises': 0, 'bets': 4, 'folds': 4746, 'posts': 0, 'checks': 0, 'straddle': 0, 'allin': 10}
corr_count: 1165


In [9]:
poker_calc = PokerProbabilityCalculator()
correct_count = 0
fold_count = 0

label_count = {action_name: 0 for action_name in ACTION_NAMES}
bot_count = {action_name: 0 for action_name in ACTION_NAMES}

corr_count = 0
for i in tqdm(random_indices):
    input_tensor, label_action_tensor, label_bet_size_tensor, baseline_inputs, game_idx = poker_dataset[i]
    call_size = baseline_inputs['call_size']
    pot_size = baseline_inputs['pot_size']
    stack_size = baseline_inputs['stack_size']
    min_raise = baseline_inputs['min_raise']
    hole_cards = baseline_inputs['hole_cards']
    community_cards = baseline_inputs['community_cards']
    num_players = baseline_inputs['num_players']
    
    poker_calc.set(hole_cards, community_cards, num_players, num_simulations = 1000)
    win_rate, lose_rate, tie_rate, avg_tied_players = poker_calc.poker_probability_calculator()
    
    action_name, bet_size = get_bet_size(win_rate, lose_rate, tie_rate, avg_tied_players, pot_size, call_size, stack_size, min_raise, tight_factor=0)
    
    # label_action_tensor is a one hot tensor
    label_action_name = ACTION_NAMES[label_action_tensor.argmax().item()]
    #print(action_name, bet_size, "\t", label_action_name)
    label_count[label_action_name] += 1
    bot_count[action_name] += 1
    
    if action_name == label_action_name:
        corr_count += 1

print('label_count:', label_count)
print('bot_count:', bot_count)
print('corr_count:', corr_count)

100%|██████████| 5000/5000 [09:37<00:00,  8.66it/s]

label_count: {'small': 0, 'big': 0, 'calls': 960, 'caps': 0, 'raises': 962, 'bets': 0, 'folds': 1032, 'posts': 0, 'checks': 1053, 'straddle': 0, 'allin': 993}
bot_count: {'small': 0, 'big': 0, 'calls': 751, 'caps': 0, 'raises': 0, 'bets': 0, 'folds': 3964, 'posts': 0, 'checks': 0, 'straddle': 0, 'allin': 285}
corr_count: 1433


In [10]:
count = 0
for i in tqdm(range(len(poker_dataset))):
    label_action_name = ACTION_NAMES[poker_dataset[i][1].argmax().item()]
    if label_action_name == 'folds':
        count += 1
        
print(count)
print(fold_count)
print(len(poker_dataset))

100%|██████████| 195875/195875 [00:01<00:00, 153348.51it/s]

39175
0
195875
